In [1]:
!pip install -U scikit-learn
!pip install xgboost
!pip install imbalanced_learn
!pip install shap
!pip install pandarallel

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

import pandas as pd
import numpy as np
from functools import reduce
import glob
import time
import datetime

now = datetime.datetime.now()

from itertools import tee, islice, chain

#Data visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.ticker as ticker
from matplotlib import colors
from matplotlib.colors import ListedColormap
from matplotlib.ticker import FixedLocator, FixedFormatter

from mpl_toolkits.mplot3d import Axes3D

import seaborn as sns
sns.set(rc={"figure.figsize":(11.7,8.27)})

import plotly.express as px
import plotly.io as pio
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode,iplot

# machine learning libraries
from imblearn.over_sampling import SMOTE
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error, accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV, StratifiedKFold, StratifiedShuffleSplit, KFold, RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import decomposition
from sklearn.impute import SimpleImputer
from numpy.random import seed
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from sklearn.ensemble import VotingClassifier
import shap

# ensemble method libraries
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

!pip install keras
# neural network libraries
import tensorflow
from tensorflow import keras
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier


         .:::.     .::.       
        ....yy:    .yy.       
        :.  .yy.    y.        
             :y:   .:         
             .yy  .:          
              yy..:           
              :y:.            
              .y.             
             .:.              
        ....:.                
        :::.                  

• Project files and data should be stored in /project. This is shared among everyone
  in the project.
• Personal files and configuration should be stored in /home/faculty.
• Files outside /project and /home/faculty will be lost when this server is terminated.
• Create custom environments to setup your servers reproducibly.


         .:::.     .::.       
        ....yy:    .yy.       
        :.  .yy.    y.        
             :y:   .:         
             .yy  .:          
              yy..:           
              :y:.            
              .y.             
             .:.              
        ....:.                
        :::.       

In [2]:
import requests

def telegram_bot_sendtext(bot_message):

    bot_token = '5544013184:AAEFue_7v4qLcmHEdS2uHBWU2hpQAczrZmU'
    bot_chatID = '1452370484'
    send_text = 'https://api.telegram.org/bot' + bot_token + '/sendMessage?chat_id=' + bot_chatID + '&parse_mode=Markdown&text=' + bot_message

    response = requests.get(send_text)

    return response.json()

In [19]:
final_data_so = pd.read_csv('final_data_so.csv', 
                      lineterminator='\n',
                      delimiter=',',
                      parse_dates=['Order_Tstamp'])

In [22]:
data_so = final_data_so

In [23]:
data_so['Order_MY'] = data_so['Order_Tstamp'].dt.strftime('%Y-%m')

In [24]:
def get_target(data):
    data['next_order'] = data.groupby('Phone')['Order_Tstamp'].shift(periods=-1)
    data['days'] = (data['next_order'] - data['Order_Tstamp']).dt.days
    data['is_recurring'] = np.where(data['days'] < 30, 1, 0)
    data = data.drop(['next_order','days'],1)
    return data

In [25]:
def get_previous(data):
    data['prev_order'] = data.groupby('Phone')['Order_Tstamp'].shift(periods=1)
    data['prev_month'] = data['prev_order'].dt.month
    return data

In [26]:
def fsum(x, t, v):
    return x.apply(lambda y: x.loc[x['Order_Tstamp'].between(y['Order_Tstamp'] - t, 
                                                     y['Order_Tstamp'],
                                                     inclusive=False),v].sum(),axis=1)

def fcnt(x, t, v):
    return x.apply(lambda y: x.loc[x['Order_Tstamp'].between(y['Order_Tstamp'] - t, 
                                                     y['Order_Tstamp'],
                                                     inclusive=False),v].count(),axis=1)

def funi(x, t, v):
    return x.apply(lambda y: x.loc[x['Order_Tstamp'].between(y['Order_Tstamp'] - t, 
                                                     y['Order_Tstamp'],
                                                     inclusive=False),v].nunique(),axis=1)

In [27]:
start_time = time.time()
data_so = get_target(data_so)
data_so = get_previous(data_so)
print("--- %s seconds ---" % (time.time() - start_time))

--- 25.880493640899658 seconds ---


In [28]:
def get_var(data):
# remove warning
    pd.options.mode.chained_assignment = None  # default='warn'
    
    # define function to get previous and next index from the list
    def previous_and_next(some_iterable):
        prevs, items, nexts = tee(some_iterable, 3)
        prevs = chain([None], prevs)
        nexts = chain(islice(nexts, 1, None), [None])
        return zip(prevs, items, nexts)

    # define period list and empty list
    mylist = ['2020-12','2021-01','2021-02','2021-03','2021-04','2021-05','2021-06','2021-07','2021-08','2021-09','2021-10','2021-11','2021-12']
    li = []

    # we create a function to iterate mylist and use for-loop to generate final data
    for previous, item, nxt in previous_and_next(mylist):
        # skip 2020Q4 and 2022Q1
        if previous == None:
            continue
        # main code
        else:  
            data = data_so[data_so['Order_MY'].isin([previous,item])].copy()
            
            temp1 = data.groupby('Phone', group_keys=False).parallel_apply(fsum, pd.Timedelta(30, unit='d'), ['Value_Taxed','Qty_Order'])
            temp1.rename(columns = {'Value_Taxed':'total_val', 'Qty_Order':'total_qty'}, inplace = True)
            
            temp2 = data.groupby('Phone', group_keys=False).parallel_apply(funi, pd.Timedelta(30, unit='d'), ['Order_Number', 'Official_Store','Principal'])
            temp2.rename(columns = {'Order_Number':'cnt_order', 'Official_Store':'cnt_os', 'Principal':'cnt_princ'}, inplace = True)
            
            
            data_frames = [data,temp1,temp2]
            df = reduce(lambda  left,right: pd.merge(left,right,left_index=True, right_index=True, how='left'), data_frames)

            df = df[df['Order_MY'] == item]
            
            li.append(df)
    
    df_final = pd.concat(li, axis=0, ignore_index=True)

    return df_final

In [29]:
start_time = time.time()
temp_df = get_var(data_so)
print("--- %s seconds ---" % (time.time() - start_time))

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The Jupyter serve

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The Jupyter serve

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The Jupyter serve

--- 12671.49848484993 seconds ---


In [30]:
def gen_feat(data):
    column_list = ['Principal', 'Principal_Categ']
    for i in (column_list):
        data = pd.get_dummies(data, columns=[i])
        
    data['current_month'] = data['Order_Tstamp'].dt.month

    data['more_than_3'] = np.where((data['cnt_order']+data['cnt_os']+data['cnt_princ']) > 3, 1, 0)

    java = ['DKI Jakarta', 'Jawa Barat', 'Banten', 'Jawa Timur', 'Jawa Tengah']
    data['in_java'] = np.where(data['Province'].isin(java), 1, 0)

    data = data.drop(columns=['Order_Tstamp', 'Status', 'SKAM', 'Warehouse', 'Buyer', 'Address', 'City', 'Postcode', 'Shipping',
                             'Marketplace', 'Official_Store', 'Gender', 'Shipping_Categ', 'Province_Categ','Province','prev_order'])

    return data

In [31]:
final_model_data = gen_feat(temp_df)

In [46]:
final_model_data.to_csv('final_model_data.csv', index=False)

In [48]:
final_model_data.shape

(5314755, 128)